## Basic setting up

In [161]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlalchemy
import datetime

In [2]:
listings = pd.read_csv("barcelona_listings.csv")

In [26]:
reviews = pd.read_csv("barcelona_reviews.csv")

In [28]:
calendar = pd.read_csv("barcelona_calendar.csv")

In [3]:
engine = create_engine("sqlite:///project.db")

In [65]:
def null_count(x):
    return listings[x].isna().sum() / len(listings[x])

def unique_count(x):
    return listings.nunique()# / listings.shape[0] * 100

#for i,c in enumerate(listings.columns):
 #   print(c, unique_count(c)[i])

## Import accommodates

In [4]:
df = pd.DataFrame(listings.accommodates.unique(), columns=["accommodates"])

In [5]:
df.to_sql("Accommodation", engine, if_exists="append", index=False, dtype={"accommodates" : sqlalchemy.CHAR(32)})

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: Accommodation.accommodates [SQL: 'INSERT INTO "Accommodation" (accommodates) VALUES (?)'] [parameters: ((6,), (8,), (2,), (3,), (4,), (5,), (10,), (1,)  ... displaying 10 of 16 total bound parameter sets ...  (14,), (15,))] (Background on this error at: http://sqlalche.me/e/gkpj)

In [ ]:
engine.execute("select count(*) from Accommodation ").fetchall()

## Import amenities

In [83]:
amenities = pd.DataFrame(listings.amenities.unique(), columns=["amenities"])

In [84]:
temp = []
for index, row in amenities.iterrows():
    temp.append(row["amenities"].split(','))

In [85]:
flat_list = []
for sublist in temp:
    for item in sublist:
        flat_list.append(item)

In [140]:
def remove_chars(item):
    for c in item:
        if c == "{" or c == "}" or c == "'" or c == '"' or c == "[" or c == "]":
            item = item.replace(c, "")
        item = item.strip().lower()
    return item

In [87]:
new_list = []
for item in flat_list:
    new_list.append('"' + remove_chars(item) + '"')
    

In [88]:
new_list = list(set(new_list))

In [90]:
new_list.remove('""')

In [91]:
new_list

['"espresso machine"',
 '"handheld shower head"',
 '"carbon monoxide detector"',
 '"stair gates"',
 '"elevator"',
 '"pets live on this property"',
 '"baby monitor"',
 '"ski-in/ski-out"',
 '"disabled parking spot"',
 '"building staff"',
 '"coffee maker"',
 '"private pool"',
 '"smart lock"',
 '"gym"',
 '"day bed"',
 '"accessible-height bed"',
 '"warming drawer"',
 '"shared pool"',
 '"microwave"',
 '"cooking basics"',
 '"wheelchair accessible"',
 '"pack ’n play/travel crib"',
 '"electric profiling bed"',
 '"free street parking"',
 '"balcony"',
 '"pillow-top mattress"',
 '"bidet"',
 '"air conditioning"',
 '"hbo go"',
 '"baby bath"',
 '"washer"',
 '"private living room"',
 '"single level home"',
 '"beachfront"',
 '"outdoor parking"',
 '"netflix"',
 '"dishes and silverware"',
 '"bath towel"',
 '"cat(s)"',
 '"sun loungers"',
 '"table corner guards"',
 '"changing table"',
 '"bbq grill"',
 '"dishwasher"',
 '"luggage dropoff allowed"',
 '"memory foam mattress"',
 '"free parking on premises"',
 '

In [92]:
amenities = pd.DataFrame(new_list, columns=["amenities"])

In [93]:
amenities.to_sql("Amenities", engine, if_exists="append", index=False, dtype={"amenities" : sqlalchemy.CHAR(32)})

In [24]:
listings.id.count()

13095

In [29]:
len(listings["amenities"].unique() == listings.amenities.unique())

11837

## Import Bathrooms

In [61]:
bathrooms = pd.DataFrame(listings.bathrooms.unique(), columns=["bathrooms"])

In [62]:
bathrooms = bathrooms.dropna()

In [63]:
bathrooms.to_sql("Bathrooms", engine, if_exists="append", index=False, dtype={"bathrooms" : sqlalchemy.FLOAT})

## Import Description

In [25]:
description = pd.DataFrame({"description_id": np.arange(len(listings.description)), "summary": listings.summary, "space": listings.space, "description": listings.description, "neighborhood_overview": listings.neighborhood_overview,"notes":listings.notes,"transit":listings.transit,"access":listings.access,"picture_url":listings.picture_url,"square_feet": listings.square_feet, "id": listings.id})

In [29]:
description["space"].fillna("",inplace=True)
description["neighborhood_overview"].fillna("",inplace=True)
description["description"].fillna("",inplace=True)
description["summary"].fillna("",inplace=True)
description["notes"].fillna("",inplace=True)
description["transit"].fillna("",inplace=True)
description["access"].fillna("",inplace=True)
description["picture_url"].fillna("",inplace=True)
description["square_feet"].fillna(-1,inplace=True)


In [32]:
description.to_sql("Described_Description", engine, if_exists="append", index=False, dtype={"description_id": sqlalchemy.INT, "summary": sqlalchemy.VARCHAR(1024), "space": sqlalchemy.VARCHAR(1024), "description": sqlalchemy.VARCHAR(1024), "neighborhood_overview": sqlalchemy.VARCHAR(1024),"notes":sqlalchemy.VARCHAR(1024),"transit": sqlalchemy.VARCHAR(1024),"access": sqlalchemy.VARCHAR(1024),"picture_url":sqlalchemy.CHAR(128),"square_feet": sqlalchemy.FLOAT, "id": sqlalchemy.INT})

## Import bedding

In [19]:
bedding = pd.DataFrame({"beds": listings.beds, "bed_type": listings.bed_type})

In [21]:
bedding = bedding.fillna(0).drop_duplicates()


In [22]:
bedding.to_sql("Bedding", engine, if_exists="append", index=False, dtype={"beds" : sqlalchemy.FLOAT, "bed_type" : sqlalchemy.CHAR(32)})

## Import Bedrooms

In [4]:
bedrooms = pd.DataFrame({"bedrooms": listings.bedrooms})

In [10]:
bedrooms = bedrooms.dropna().drop_duplicates()

In [11]:
bedrooms

,bedrooms
0,2.0
1,3.0
2,1.0
12,4.0
77,0.0
97,5.0
261,16.0
308,6.0
1436,12.0
2312,8.0


In [12]:
bedrooms.astype(int)

,bedrooms
0,2
1,3
2,1
12,4
77,0
97,5
261,16
308,6
1436,12
2312,8


In [13]:
bedrooms.to_sql("Bedrooms", engine, if_exists="append", index=False, dtype={"bedrooms" : sqlalchemy.INT})

## Import Room

In [14]:
room = pd.DataFrame({"room_type": listings.room_type})

In [17]:
room = room.drop_duplicates()

In [18]:
room.to_sql("Room", engine, if_exists="append", index=False, dtype={"room_type" : sqlalchemy.CHAR(32)})

## Import Property

In [19]:
prop = pd.DataFrame({"property_type": listings.property_type})

In [23]:
prop = prop.drop_duplicates()

In [24]:
prop.to_sql("Property", engine, if_exists="append", index=False, dtype={"propery_type" : sqlalchemy.CHAR(32)})

## Import Reviewer

In [29]:
reviewer = pd.DataFrame({"reviewer_id": reviews.reviewer_id, "reviewer_name" : reviews.reviewer_name})

In [50]:
reviewer = reviewer.drop_duplicates().drop(174201)

In [51]:
reviewer.to_sql("Reviewer", engine, if_exists="append", index=False, dtype={"reviewer_id" : sqlalchemy.INT, "reviewer_name" : sqlalchemy.CHAR(32)})

In [49]:
reviewer[reviewer.duplicated(['reviewer_id'], keep=False)]

,reviewer_id,reviewer_name
174201,6148697,Casa Nuna
179938,6148697,Mi Casa Bali


## Import Reviewed TODO

In [54]:
reviewed = pd.DataFrame({"id": reviews.listing_id, "reviewer_id" : reviews.reviewer_id, "comments": reviews.comments, "review_date" : reviews.date})

In [55]:
reviewed

,id,reviewer_id,comments,review_date
0,18666,1559265,Very nice flat. We had easy transportation to ...,2015-10-10
1,18674,4841196,"Great location. Clean, spacious flat. Would re...",2013-05-27
2,18674,11600277,Mi mejor recomendación para este departamento....,2014-03-02
3,18674,35231385,"Big apartment, well equipped.\nVery good servi...",2015-08-04
4,18674,23223644,The Check in was fast and flexible. The price ...,2016-06-20
5,18674,4756672,Great location and enough space in the apartme...,2018-06-18
6,21605,57647372,Meritxell était très accueillante et nous a mi...,2016-05-08
7,21605,62165990,Con mi esposa nos hospedamos en casa de Michae...,2016-05-22
8,21605,67778905,Even though the hosts were not there at the mo...,2016-05-27
9,21605,64869468,La habitación es perfecta para una pareja y la...,2016-05-29


In [58]:
reviewed[reviewed.duplicated(['id', "review_date", "reviewer_id"], keep=False)]

,id,reviewer_id,comments,review_date
134162,1996730,85117476,"El apartamiento es increible, muy espacioso y ...",2016-07-23
134163,1996730,85117476,La segunda vez que me quedo en su piso y como ...,2016-07-23
210431,3823973,24495890,The reservation was canceled 17 days before ar...,2015-03-08
210796,3823973,24495890,The reservation was canceled 24 days before ar...,2015-03-08
212882,3823973,24495890,The reservation was canceled 10 days before ar...,2015-03-08
263426,6449766,57829170,The host canceled this reservation 24 days bef...,2018-06-05
263427,6449766,57829170,The host canceled this reservation 33 days bef...,2018-06-05
378123,16347842,19641299,The host canceled this reservation 11 days bef...,2017-11-16
378124,16347842,19641299,The host canceled this reservation 7 days befo...,2017-11-16


#### We need to change the primary key to (id, reviewer_id, date) and decide what to do with the duplicates

## Import Pricing

In [118]:
pricing = pd.DataFrame({"price": listings.price, "weekly_price" : listings.weekly_price, "monthly_price": listings.monthly_price, "security_deposit" : listings.security_deposit, "cleaning_fee" : listings.cleaning_fee, "guests_included" : listings.guests_included, "extra_people" : listings.extra_people, "minimum_nights" : listings.minimum_nights, "maximum_nights" : listings.maximum_nights, "id" : listings.id})

In [119]:
for i in pricing.index:
    pricing.at[i, "price"] = float(str(pricing.at[i, "price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "weekly_price"]):
        pricing.at[i, "weekly_price"] = float(str(pricing.at[i, "weekly_price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "monthly_price"]):
        pricing.at[i, "monthly_price"] = float(str(pricing.at[i, "monthly_price"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "security_deposit"]):
        pricing.at[i, "security_deposit"] = float(str(pricing.at[i, "security_deposit"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "cleaning_fee"]):
        pricing.at[i, "cleaning_fee"] = float(str(pricing.at[i, "cleaning_fee"])[1:].replace(",", ""))
    if not pd.isnull(pricing.at[i, "extra_people"]):
        pricing.at[i, "extra_people"] = float(str(pricing.at[i, "extra_people"])[1:].replace(",", ""))

In [120]:
for i in pricing.index:
    if pd.isnull(pricing.at[i, "weekly_price"]):
        pricing.at[i, "weekly_price"] = pricing.at[i, "price"] * 7
    if pd.isnull(pricing.at[i, "monthly_price"]):
        pricing.at[i, "monthly_price"] = pricing.at[i, "price"] * 30
    if pd.isnull(pricing.at[i, "security_deposit"]):
        pricing.at[i, "security_deposit"] = 0
    if pd.isnull(pricing.at[i, "cleaning_fee"]):
        pricing.at[i, "cleaning_fee"] = 0

In [123]:
pricing.to_sql("Pricing", engine, if_exists="append", index=False, dtype={"price": sqlalchemy.FLOAT, "weekly_price" : sqlalchemy.FLOAT, "monthly_price": sqlalchemy.FLOAT, "security_deposit" : sqlalchemy.FLOAT, "cleaning_fee" : sqlalchemy.FLOAT, "guests_included" : sqlalchemy.INT, "extra_people" : sqlalchemy.FLOAT, "minimum_nights" : sqlalchemy.INT, "maximum_nights" : sqlalchemy.INT, "id" : sqlalchemy.INT})

## Import Verification

In [129]:
verification = pd.DataFrame(listings.host_verifications.unique(), columns=["host_verifications"])

In [133]:
temp = []
for index, row in verification.iterrows():
    temp.append(row["host_verifications"].split(','))

In [135]:
flat_list = []
for sublist in temp:
    for item in sublist:
        flat_list.append(item)

In [141]:
new_list = []
for item in flat_list:
    new_list.append('"' + remove_chars(item) + '"')

In [142]:
new_list = list(set(new_list))

In [147]:
new_list.remove('""')

In [148]:
new_list

['"google"',
 '"sesame"',
 '"phone"',
 '"government_id"',
 '"zhima_selfie"',
 '"jumio"',
 '"facebook"',
 '"photographer"',
 '"kba"',
 '"manual_offline"',
 '"offline_government_id"',
 '"selfie"',
 '"weibo"',
 '"work_email"',
 '"reviews"',
 '"sent_id"',
 '"sesame_offline"',
 '"manual_online"',
 '"identity_manual"',
 '"email"']

In [151]:
verification = pd.DataFrame({"host_verifications_id" : np.arange(len(new_list)), "host_verifications" : new_list})

In [152]:
verification.to_sql("Verification", engine, if_exists="append", index=False, dtype={"host_verifications_id" : sqlalchemy.INT, "host_verifications" : sqlalchemy.CHAR(32)})

## Import verified_by TODO

## Import Calendar

In [169]:
cal = pd.DataFrame({"cal_id" : np.arange(len(calendar.date.unique())), "date" : calendar.date.unique()})

In [171]:
for i in cal.index:
    cal.at[i, "date"] = datetime.datetime.strptime(str(cal.at[i, "date"]), "%Y-%m-%d").date()

In [173]:
cal.to_sql("Calendar", engine, if_exists="append", index=False, dtype={"cal_id" : sqlalchemy.INT, "date" : sqlalchemy.DATE})

## Import Available_at

In [191]:
date_mapping = {}
for i in cal.index:
    date_mapping[str(cal.at[i, "date"])] = cal.at[i, "cal_id"]

In [193]:
second_mapping = []
for i in calendar.index:
    second_mapping.append(date_mapping.get(calendar.at[i, "date"]))

In [212]:
avail = pd.DataFrame({"id" : calendar.listing_id, "cal_id" : second_mapping, "price" : calendar.price, "available" : calendar.available})

In [215]:
for i in avail.index:
    if not pd.isna(avail.at[i, "price"]):
        avail.at[i, "price"] = float(str(avail.at[i, "price"]).replace(",","").replace("$", ""))

In [217]:
avail = avail.fillna(-1)

In [219]:
avail.to_sql("Available_at", engine, if_exists="append", index=False, dtype={"id" : sqlalchemy.INT, "cal_id" : sqlalchemy.INT, "price" : sqlalchemy.INT, "available" : sqlalchemy.CHAR(1)})